In [ ]:
import torch
import torchvision
import torch.nn as nn

In [ ]:
class ResNetBlock(nn.Module):
    def __init__(self):
        super().__init__()
        pass

    def forward(self, x):
        pass

In [ ]:
class ResNet18(nn.Module):
    def __init__(self):
        super().__init__()
        pass

    def forward(self, x):
        pass

In [ ]:
lossfn = nn.CrossEntropyLoss()
num_epochs = 10
net = ResNet18()
optim = torch.optim.SGD(net.parameters, lr=1e-3)

for i in range(num_epochs):
    epoch_loss = 0
    for x, y in dataloader:
        y_hat = net(x)
        loss = lossfn(y_hat, y)
        optim.zero_grad()
        loss.backward()
        optim.step()
        epoch_loss += loss
    print("Loss on epoch {i} was {epoch_loss}")
